In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow as tf
import os
import time
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab import drive

!wget -q https://git.io/JYx2x -O resnet_cifar10_v2.py
import resnet_cifar10_v2

drive.mount('/content/drive/')
loc = 'drive/MyDrive/Shtem2023/'
print(os.listdir(loc+'/all_data'))

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
['breastcancerX.npy', 'breastcancerY.npy', 'alzheimerY.npy', 'alzheimerX.npy', 'covidX.npy', 'covidY.npy', 'cifar10X.npy', 'cifar10Y.npy', 'glaucomaX.npy', 'glaucomaY.npy', 'applescabX.npy', 'applescabY.npy']


Sourced from https://keras.io/examples/vision/simsiam/

In [ ]:

##########    THIS IS THE ONLY CELL YOU NEED TO CHANGE :) ################

dataset_name = 'cifar10'
X = np.load(loc+'/all_data/'+dataset_name+'X.npy')
Y = np.load(loc+'/all_data/'+dataset_name+'Y.npy')
littleNtrain = 128 # make sure Ntrain + Ntest < dataset size!!!!! # WANT NTRAIN ~ 200-1000
num_cat = 10
CROP_TO = 32

In [ ]:
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 8 # maybe up to 256 # lol no, don't change from 8
Ntest = 208 # make sure Ntrain + Ntest < dataset size!!!!! # WANT NTRAIN ~ 200-1000

EPOCHS = 15

SEED = 26
PROJECT_DIM = 128
LATENT_DIM = 64
WEIGHT_DECAY = 0.0005
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
t0 = time.time()
for littleNtrain in np.arange(8,201,8):
  x_train, y_train, x_test, y_test  = X[:littleNtrain], Y[:littleNtrain], X[-Ntest:], Y[-Ntest:]

  class SimSiam(tf.keras.Model):
      def __init__(self, encoder, predictor):
          super().__init__()
          self.encoder = encoder
          self.predictor = predictor
          self.loss_tracker = tf.keras.metrics.Mean(name="loss")

      @property
      def metrics(self):
          return [self.loss_tracker]

      def train_step(self, data):
          # Unpack the data.
          ds_one, ds_two = data

          # Forward pass through the encoder and predictor.
          with tf.GradientTape() as tape:
              z1, z2 = self.encoder(ds_one), self.encoder(ds_two)
              p1, p2 = self.predictor(z1), self.predictor(z2)
              # Note that here we are enforcing the network to match
              # the representations of two differently augmented batches
              # of data.
              loss = compute_loss(p1, z2) / 2 + compute_loss(p2, z1) / 2

          # Compute gradients and update the parameters.
          learnable_params = (
              self.encoder.trainable_variables + self.predictor.trainable_variables
          )
          gradients = tape.gradient(loss, learnable_params)
          self.optimizer.apply_gradients(zip(gradients, learnable_params))

          # Monitor loss.
          self.loss_tracker.update_state(loss)
          return {"loss": self.loss_tracker.result()}

  N = 2
  DEPTH = N * 9 + 2
  NUM_BLOCKS = ((DEPTH - 2) // 9) - 1


  def get_encoder():
      # Input and backbone.
      inputs = layers.Input((CROP_TO, CROP_TO, 3))
      x = layers.Rescaling(scale=1.0 / 127.5, offset=-1)(
          inputs
      )
      x = resnet_cifar10_v2.stem(x)
      x = resnet_cifar10_v2.learner(x, NUM_BLOCKS)
      x = layers.GlobalAveragePooling2D(name="backbone_pool")(x)

      # Projection head.
      x = layers.Dense(
          PROJECT_DIM, use_bias=False, kernel_regularizer=regularizers.l2(WEIGHT_DECAY)
      )(x)
      x = layers.BatchNormalization()(x)
      x = layers.ReLU()(x)
      x = layers.Dense(
          PROJECT_DIM, use_bias=False, kernel_regularizer=regularizers.l2(WEIGHT_DECAY)
      )(x)
      outputs = layers.BatchNormalization()(x)
      return tf.keras.Model(inputs, outputs, name="encoder")


  def get_predictor():
      model = tf.keras.Sequential(
          [
              # Note the AutoEncoder-like structure.
              layers.Input((PROJECT_DIM,)),
              layers.Dense(
                  LATENT_DIM,
                  use_bias=False,
                  kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
              ),
              layers.ReLU(),
              layers.BatchNormalization(),
              layers.Dense(PROJECT_DIM),
          ],
          name="predictor",
      )
      return model

      ###  TRAINING WITHOUT SELF SUPERVISED LEARNING

  simsiam = SimSiam(get_encoder(), get_predictor())

  # Create a cosine decay learning scheduler.
  num_training_samples = len(x_train)
  steps = EPOCHS * (num_training_samples // BATCH_SIZE)
  lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecay(
      initial_learning_rate=0.03, decay_steps=steps
  )

  # Create an early stopping callback.
  early_stopping = tf.keras.callbacks.EarlyStopping(
      monitor="loss", patience=15, restore_best_weights=True
  )

  def self_less_model(indim):
    inputs = layers.Input((indim, indim, 3))
    resnet = tf.keras.Model(
      simsiam.encoder.input, simsiam.encoder.get_layer("backbone_pool").output
    )
    # We then create our linear classifier and train it.
    x = resnet(inputs)
    outputs = layers.Dense(num_cat, activation="softmax")(x)
    linear_model = tf.keras.Model(inputs, outputs, name="linear_model")
    # Compile model and start training.
    return linear_model

  self_less = self_less_model(x_train[0].shape[0])
  self_less.trainable=True
  self_less.summary()
  self_less.compile(
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"],
      optimizer=tf.keras.optimizers.SGD(lr_decayed_fn, momentum=0.9),
  )

  history = self_less.fit(x_train,y_train, epochs=EPOCHS, validation_data=(x_test, y_test))
  pd.DataFrame(history.history).to_csv(loc + "Results/selfLess_"  + str(littleNtrain) + "_" + dataset_name + ".csv")
  print('SELF LESS TRAINING RESULTS SAVED!!!\n'*4)
  print("\n", np.round(time.time() - t0,2), "\nlittleNtrain",littleNtrain, "\n")

Model: "linear_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 model_7 (Functional)        (None, 256)               568368    
                                                                 
 dense_35 (Dense)            (None, 10)                2570      
                                                                 
Total params: 570,938
Trainable params: 567,930
Non-trainable params: 3,008
_________________________________________________________________
Epoch 1/15
1/1 [==============================] - 5s 5s/step - loss: 3.8737 - accuracy: 0.0000e+00 - val_loss: 3.8459 - val_accuracy: 0.0769
Epoch 2/15
1/1 [==============================] - 1s 626ms/step - loss: 2.9691 - accuracy: 0.5000 - val_loss: 3.8482 - val_accuracy: 0.0962
Epoch 3/15
1/1 [===========